In [1]:
import json
import pandas as pd
#import spacy
#nlp = spacy.load("ru_core_news_sm")

In [2]:
import os

In [3]:
df = pd.read_csv('data/for labeling/results-2.tsv', sep='\t')

In [4]:
len(df)

200

In [5]:
groups = df.groupby('INPUT:title')

In [6]:
for name, group in groups:
    print(name)

12 стульев
Леди Макбет Мценского уезда
Мастер и Маргарита
Обломов
Собачье сердце


In [7]:
symbols = (u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ",
           u"abvgdeejzijklmnoprstufhzcss_y_euaABVGDEEJZIJKLMNOPRSTUFHZCSS_Y_EUA")
tr = {ord(a):ord(b) for a, b in zip(*symbols)}

In [8]:
name.translate(tr).replace(' ', '_')

'Sobac_e_serdze'

In [22]:
os.makedirs('data/brat')

In [14]:
for name, group in groups:
    with open(os.path.join('data', 'brat', name.translate(tr).replace(' ', '_').lower() + '.txt'), 'w') as f:
        text = '\r\n'.join(group.sort_values('INPUT:id', axis=0)['INPUT:text'].unique())
        f.write(text)

In [11]:
!ls data/brat

12_stul_ev.ann                  oblomov.ann
12_stul_ev.txt                  oblomov.txt
anna_karenina.ann               otza_i_deti.ann
anna_karenina.txt               otza_i_deti.txt
geroj_nasego_vremeni.ann        prestuplenie_i_nakazanie.ann
geroj_nasego_vremeni.txt        prestuplenie_i_nakazanie.txt
ledi_makbet_mzenskogo_uezda.ann sobac_e_serdze.ann
ledi_makbet_mzenskogo_uezda.txt sobac_e_serdze.txt
master_i_margarita.ann          vojna_i_mir.ann
master_i_margarita.txt          vojna_i_mir.txt


In [12]:
group.iloc[3]['OUTPUT:result']

'{"label":"per"\\,"offset":743\\,"length":4\\,"segment":"Влас"},{"label":"pro"\\,"offset":772\\,"length":2\\,"segment":"он"},{"label":"per"\\,"offset":888\\,"length":4\\,"segment":"Влас"},{"label":"pro"\\,"offset":934\\,"length":3\\,"segment":"ему"},{"label":"per_oblique"\\,"offset":972\\,"length":8\\,"segment":"личность"},{"label":"per_oblique"\\,"offset":990\\,"length":5\\,"segment":"повар"}'

In [16]:
#T0	PER 287 297	a stranger
for name, group in groups:
    cur_mention = 0
    group = group.sort_values('INPUT:id', axis=0)
    fulltext = '\r\n'.join(group.sort_values('INPUT:id', axis=0)['INPUT:text'].unique())
    offsets = []
    with open(os.path.join('brat-1.3p1', 'data', name.translate(tr).replace(' ', '_').lower() + '.ann'), 'w') as f:
        for ind, subgroup in group.groupby('INPUT:id'):
            if not offsets:
                cur_offset = 0
            else:
                cur_offset = offsets[-1]
            par_annotations = []
            for _, row in subgroup.iterrows():
                if not pd.isna(row['OUTPUT:result']) and row['ASSIGNMENT:status'] != 'REJECTED':
                    par_annotations.extend(json.loads('[' + row['OUTPUT:result'].replace('\\', '') + ']'))
            par_annotations = sorted(par_annotations, key=lambda x: x['offset'])
            for annotation in par_annotations:
                start = cur_offset+annotation['offset']
                end = cur_offset+annotation['offset']+annotation['length']
                text = fulltext[start:end]
                if annotation['label'] == 'per':
                    f.write('\t'.join([f'T{cur_mention}', f'PROP_PER {start} {end}', text]))
                elif annotation['label'] == 'per_oblique':
                    f.write('\t'.join([f'T{cur_mention}', f'NOM_PER {start} {end}', text]))
                elif annotation['label'] == 'pro':
                    f.write('\t'.join([f'T{cur_mention}', f'PRON_PER {start} {end}', text]))
                f.write('\n')
                cur_mention += 1
            offsets.append(len(row['INPUT:text'])+cur_offset+2)